In [1]:
# for Google Colab only: Mount my Google Drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
pwd

'/gdrive'

In [3]:
# for Google Colab only: Navigate to my snaug project folder
cd 'My Drive/Colab Notebooks/snaug'

/gdrive/My Drive/Colab Notebooks/snaug


In [4]:
# peek at current system Path settings
import sys
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [5]:
# let's add my project path so that I can load my custom written
# library functions saved in '/lib' 
sys.path.append("/gdrive/My Drive/Colab Notebooks/snaug")
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/gdrive/My Drive/Colab Notebooks/snaug']

In [6]:
#
# Load saved weights of models that were previously trained on cloud platforms 
# using GPU (eg. Google Colab)
# Save an instance of trained final model for productionalisation
#

import string
import textwrap
import pickle

from lib.nlplstm_class import TFModelLSTMWord2vec 
from lib.data_common import (load_doc, save_doc, clean_doc, prepare_char_tokens)
from lib.data_common import (build_token_lines, prepare_text_tokens, load_word2vec)
from lib.data_common import pathfinder_textfile, fixed_length_token_textfile

Using TensorFlow backend.


In [16]:
%%time
# load fixed-length lines of tokens
doc = load_doc(fixed_length_token_textfile)
lines = doc.split('\n')
#print('Total lines: %d' % len(lines))

CPU times: user 4.53 ms, sys: 4.21 ms, total: 8.74 ms
Wall time: 19.9 ms


In [17]:
%%time
# tokenize and separate into features and target
X, y, seq_length, vocab_size, tokenizer = prepare_text_tokens(lines)
#print(X.shape)

CPU times: user 853 ms, sys: 55.9 ms, total: 909 ms
Wall time: 919 ms


In [18]:
%%time
# load gensim Word2Vec word model's pretrained weights
pretrained_weights = pickle.load(open('./model/pathfinder_wordtoken_w2v_word_model_weights.pkl', 'rb'))
vocab_size, emdedding_size = pretrained_weights.shape

CPU times: user 4.34 ms, sys: 1.32 ms, total: 5.65 ms
Wall time: 11.4 ms


In [0]:
# create new object that is an LSTM model using word tokenization
# and pre-trained Word2vec model form Gensim to generate text
# this model does not use CudaDNN library
#
textgen_final_model = TFModelLSTMWord2vec(use_gpu=False)

In [20]:
# define the model parameters
textgen_final_model.define(vocab_size=vocab_size, 
                       embedding_size=emdedding_size, 
                       pretrained_weights=pretrained_weights)
print(textgen_final_model.model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 300)         877500    
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 300)         721200    
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 300)         0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dropout_4 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_4 (Dense)              (None, 2926)             

In [21]:
%%time
# load model weights trained on platform using GPU
textgen_final_model.load_weights("./model/pathfinder_wordtoken_w2v_model_50_epoch")

CPU times: user 153 ms, sys: 23.9 ms, total: 177 ms
Wall time: 198 ms


In [22]:
%%time
# save model updated with previously trained model weights
textgen_final_model.save("./model/pathfinder_wordtoken_w2v_model_50_epoch_noncuda")

CPU times: user 59 ms, sys: 12.5 ms, total: 71.5 ms
Wall time: 466 ms
